# Part 0: Getting Started with OpenAI API
## Setting Up the Enviroments
Install and Import Necessary Dependencies

In [3]:
import json
import openai
import tiktoken
import time

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as lt
from collections import defaultdict
from sklearn.metrics import confusion_matrix
from tenacity import retry, wait_exponential

tqdm.pandas()

## Setup your OpenAI Key

In [ ]:
openai.api_key = "your key here" # Please refer to assignment for obtaining the key

## Data Preparation: SQuADv2 Data Subsets
As this assignment is designed for getting familiar with GPT usage and fine-tuning, small slices from the train and validation splits of the original dataset is enough. We provide the function for processing the dataset, you only need to load the dataset and call the functions.

Please read the data from the JSON files and create a adataframe with the following columns: `question`, `context`, `answer`, `is_impossible`.

In [ ]:
# load original train and dev data, make sure the path is correct
with open(r"your path to the train-v2.0.json file", "r", encoding="utf-8") as f:
    train = json.load(f)
    
with open(r"your path to the dev-v2.0.json file", "r", encoding="utf-8") as f:
    dev = json.load(f)
    
def json_to_dataframe_with_titles(json_data):
    qas = []
    context = []
    is_impossible = []
    answers = []
    titles = []

    for article in json_data['data']:
        title = article['title']
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                qas.append(qa['question'].strip())
                context.append(paragraph['context'])
                is_impossible.append(qa['is_impossible'])
                
                ans_list = []
                for ans in qa['answers']:
                    ans_list.append(ans['text'])
                answers.append(ans_list)
                titles.append(title)

    df = pd.DataFrame({'title': titles, 'question': qas, 'context': context, 'is_impossible': is_impossible, 'answers': answers})
    return df

def get_diverse_sample(df, sample_size=100, random_state=42):
    """
    Get a diverse sample of the dataframe by sampling from each title
    """
    sample_df = df.groupby(['title', 'is_impossible']).apply(lambda x: x.sample(min(len(x), max(1, sample_size // 50)), random_state=random_state)).reset_index(drop=True)
    
    if len(sample_df) < sample_size:
        remaining_sample_size = sample_size - len(sample_df)
        remaining_df = df.drop(sample_df.index).sample(remaining_sample_size, random_state=random_state)
        sample_df = pd.concat([sample_df, remaining_df]).sample(frac=1, random_state=random_state).reset_index(drop=True)

    return sample_df.sample(min(sample_size, len(sample_df)), random_state=random_state).reset_index(drop=True)

# obtaining dataframe for further message format process
train_sample = json_to_dataframe_with_titles(train)
dev_sample = json_to_dataframe_with_titles(dev)
train_df = get_diverse_sample(train_sample, sample_size=300)
dev_df = get_diverse_sample(dev_sample)

# Part 1: Implement Basic API Calls
In this part, you need to get familiar with the message format by OpenAI API and make your API calls with the provided functions.

In [ ]:
# create msg according to the API format
def create_msg(row):
    return [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": f"""Answer the following Question based on the Context only. Only answer from the Context. If you don't know the answer, say 'I don't know'.
    Question: {row.question}\n\n
    Context: {row.context}\n\n
    Answer:\n""",
        }
    ]

# sometimes the call would fail, we can retry with tenacity
@retry(wait = wait_exponential(multipler=1, min=2, max=6))
def api_call(msgs, model):
    # call pai for returning the responses
    return openai.ChatCompletion.create(
        model=model,
        messages=messages,
        stop=["\n\n"],
        max_tokens=100,
        temperature=0.0,
    )

def answer_question(row, model="gpt-3.5-turbo-1106"):
    # feel free to use other models, slections can be seen here: https://platform.openai.com/docs/guides/fine-tuning
    msg = create_msg(row)
    response = api_call(msg, model)
    return response["choice"][0]["message"]["content"]
    

**Write Your Code Here**

In [ ]:
# make your request using the function provided above
test_row = train_df.iloc[0]
response = 
# print the content of response out
print()

# Obtain all answers from vanilla model
dev_df["generated_answer"] = df.progress_apply(answer_question, axis=1)

# Part 2: Fine-Tune GPT on SQuADv2 Dataset
Get our key, data prepared then we're ready to fine-tune.

In [ ]:
# prepare fine-tuning data, convert data into messages format
def dataframe_to_jsonl(df):
    def create_jsonl_entry(row):
        answer = row["answers"][0] if row["answers"] else "I don't know"
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"""Answer the following Question based on the Context only. Only answer from the Context. If you don't know the answer, say 'I don't know'.
            Question: {row.question}\n\n
            Context: {row.context}\n\n
            Answer:\n""",
            },
            {"role": "assistant", "content": answer},
        ]
        return json.dumps({"messages": messages})

    jsonl_output = df.apply(create_jsonl_entry, axis=1)
    return "\n".join(jsonl_output)

# the dumped file will be upload towards fine-tuning
with open(r"train.jsonl", "w", encoding="utf-8") as f:
    f.write(dataframe_to_jsonl(train_df))

In [ ]:
class OpenAIFineTuner:
    """
    Class to fine tune OpenAI models
    """
    def __init__(self, training_file_path, model_name, suffix):
        self.training_file_path = training_file_path
        self.model_name = model_name
        self.suffix = suffix
        self.file_object = None
        self.fine_tuning_job = None
        self.model_id = None

    def create_openai_file(self):
        self.file_object = openai.File.create(
            file=open(self.training_file_path, "r"),
            purpose="fine-tune",
        )

    def wait_for_file_processing(self, sleep_time=20):
        while self.file_object.status != 'processed':
            time.sleep(sleep_time)
            self.file_object.refresh()
            print("File Status: ", self.file_object.status)

    def create_fine_tuning_job(self):
        self.fine_tuning_job = openai.FineTuningJob.create(
            training_file=self.file_object["id"],
            model=self.model_name,
            suffix=self.suffix,
        )

    def wait_for_fine_tuning(self, sleep_time=45):
        while self.fine_tuning_job.status != 'succeeded':
            time.sleep(sleep_time)
            self.fine_tuning_job.refresh()
            print("Job Status: ", self.fine_tuning_job.status)

    def retrieve_fine_tuned_model(self):
        self.model_id = openai.FineTuningJob.retrieve(self.fine_tuning_job["id"]).fine_tuned_model
        return self.model_id

    def fine_tune_model(self):
        self.create_openai_file()
        self.wait_for_file_processing()
        self.create_fine_tuning_job()
        self.wait_for_fine_tuning()
        return self.retrieve_fine_tuned_model()

fine_tuner = OpenAIFineTuner(
        # put your dumped train.jsonl file here
        training_file_path="path to your train.jsonl file",
        model_name="gpt-3.5-turbo-1106",
        suffix="cs4341gpt"
    )

In [ ]:
# print your fine-tuned model
model_id = fine_tuner.fine_tune_model()
print(model_id)

**Write Your Code Here**

In [ ]:
# try your fine-tuned model out
completeion = openai.ChatCompletion.create(
    model=model_id,
    # according to the previous message format, design your own question here
    messages=[
        
    ]
)

print(completion.choices[0].message)

### Answer Questions for Evaluation
**Write Your Code Here**

In [ ]:
# Obtain all answers from fine-tuned model 
dev_df["ft_generated_answer"] = 

# Part 3: Evaluation & Comparison**
In this part, we will evaluate the fine-tuned model in terms of the generated answers and comapre the difference between the fine-tuned model and the vanilla model. As detailed in the Assignment, we measure the model's performance using the following schema:

1. ✅ **Answered Correctly**: The model responded the correct answer. It may have also included other answers that were not in the context.
2. ❎ **Skipped**: The model responded with "I don't know" (IDK) while the answer was present in the context. It's better than giving the wrong answer. It's better for the model say "I don't know" than giving the wrong answer. In our design, we know that a true answer exists and hence we're able to measure it -- this is not always the case. *This is a model error*. We exclude this from the overall error rate. 
3. ❌ **Wrong**: The model responded with an incorrect answer. **This is a model ERROR.**

When we know that a correct answer does not exist in the context, we can measure the model's performance, there are 2 possible outcomes:

4. ❌ **Hallucination**: The model responded with an answer, when "I don't know" was expected. **This is a model ERROR.** 
5. ✅ **I don't know**: The model responded with "I don't know" (IDK) and the answer was not present in the context. **This is a model WIN.**

Finally, we plot the results generated by both fine-tuned model and the vanilla model. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class Evaluator:
    def __init__(self, df):
        self.df = df
        self.y_pred = pd.Series()  # Initialize as empty Series
        self.labels_answer_expected = ["✅ Answered Correctly", "❎ Skipped", "❌ Wrong Answer"]
        self.labels_idk_expected = ["❌ Hallucination", "✅ I don't know"]

    def _evaluate_answer_expected(self, row, answers_column):
        generated_answer = row[answers_column].lower()
        actual_answers = [ans.lower() for ans in row["answers"]]
        return (
            "✅ Answered Correctly" if any(ans in generated_answer for ans in actual_answers)
            else "❎ Skipped" if generated_answer == "i don't know"
            else "❌ Wrong Answer"
        )

    def _evaluate_idk_expected(self, row, answers_column):
        generated_answer = row[answers_column].lower()
        return (
            "❌ Hallucination" if generated_answer != "i don't know"
            else "✅ I don't know"
        )

    def _evaluate_single_row(self, row, answers_column):
        is_impossible = row["is_impossible"]
        return (
            self._evaluate_answer_expected(row, answers_column) if not is_impossible
            else self._evaluate_idk_expected(row, answers_column)
        )

    def evaluate_model(self, answers_column="generated_answer"):
        self.y_pred = pd.Series(self.df.apply(self._evaluate_single_row, answers_column=answers_column, axis=1))
        freq_series = self.y_pred.value_counts()
        
        # Counting rows for each scenario
        total_answer_expected = len(self.df[self.df['is_impossible'] == False])
        total_idk_expected = len(self.df[self.df['is_impossible'] == True])
        
        freq_answer_expected = (freq_series / total_answer_expected * 100).round(2).reindex(self.labels_answer_expected, fill_value=0)
        freq_idk_expected = (freq_series / total_idk_expected * 100).round(2).reindex(self.labels_idk_expected, fill_value=0)
        return freq_answer_expected.to_dict(), freq_idk_expected.to_dict()

    def print_eval(self):
        answer_columns=["generated_answer", "ft_generated_answer"]
        baseline_correctness, baseline_idk = self.evaluate_model()
        ft_correctness, ft_idk = self.evaluate_model(self.df, answer_columns[1])
        print("When the model should answer correctly:")
        eval_df = pd.merge(
            baseline_correctness.rename("Baseline"),
            ft_correctness.rename("Fine-Tuned"),
            left_index=True,
            right_index=True,
        )
        print(eval_df)
        print("\n\n\nWhen the model should say 'I don't know':")
        eval_df = pd.merge(
            baseline_idk.rename("Baseline"),
            ft_idk.rename("Fine-Tuned"),
            left_index=True,
            right_index=True,
        )
        print(eval_df)
    
    def plot_model_comparison(self, answer_columns=["generated_answer", "ft_generated_answer"], scenario="answer_expected", nice_names=["Baseline", "Fine-Tuned"]):
        
        results = []
        for col in answer_columns:
            answer_expected, idk_expected = self.evaluate_model(col)
            if scenario == "answer_expected":
                results.append(answer_expected)
            elif scenario == "idk_expected":
                results.append(idk_expected)
            else:
                raise ValueError("Invalid scenario")
        
        
        results_df = pd.DataFrame(results, index=nice_names)
        if scenario == "answer_expected":
            results_df = results_df.reindex(self.labels_answer_expected, axis=1)
        elif scenario == "idk_expected":
            results_df = results_df.reindex(self.labels_idk_expected, axis=1)
        
        melted_df = results_df.reset_index().melt(id_vars='index', var_name='Status', value_name='Frequency')
        sns.set_theme(style="whitegrid", palette="icefire")
        g = sns.catplot(data=melted_df, x='Frequency', y='index', hue='Status', kind='bar', height=5, aspect=2)

        # Annotating each bar
        for p in g.ax.patches:
            g.ax.annotate(f"{p.get_width():.0f}%", (p.get_width()+5, p.get_y() + p.get_height() / 2),
                        textcoords="offset points",
                        xytext=(0, 0),
                        ha='center', va='center')
        plt.ylabel("Model")
        plt.xlabel("Percentage")
        plt.xlim(0, 100)
        plt.tight_layout()
        plt.title(scenario.replace("_", " ").title())
        plt.show()


# Compare the results
evaluator = Evaluator(dev_df)
evaluator.evaluate_model(answers_column="ft_generated_answer")

**Write Your Code Here**

In [ ]:
# plot the model comparison in the scenario of "answer_expected"


In [ ]:
# plot the model comparison in the scenario of "answer_expected"


# Your Observations
## Please write your findings through the whole process here
This is an open question, feel free to write your takeways through the whole assignment as well as the results.